## Integrating rigorous desalination models into network optimization
It is often desired to design a desalination plant at a certain desalination site in the network. However, it is not clear what the optimal design and operation of the plant should be. In this notebook we show how to co-optimize the network operation and desalination design by using the integrated optimization model available in PARETO. 

First we import the necessary utilities to read the network data, the function to build the integrated model with MEE-MVR as the desalination model choice (`integrated_model_build`) and the Pyomo package.

In [1]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2024 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from importlib import resources
from pareto.utilities.get_data import get_data
from pareto.models_extra.CM_module.cm_utils.data_parser import data_parser
from pareto.models_extra.integrate_desal.integrated_models.integrated_optimization_mvr import (
    integrated_model_build,
)
import pyomo.environ as pyo

Importing network data

In [2]:
with resources.path(
    "pareto.models_extra.CM_module.case_studies",
    "CM_integrated_desalination_demo.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, model_type="critical_mineral")

    network_data = data_parser(df_sets, df_parameters)

C:\Users\Tanner\AppData\Local\Temp\ipykernel_29276\4187180339.py:5: UserWarning: Invalid PARETO input has been provided. Check that the input tab names match valid PARETO input. If you'd like to read custom tabs (e.g., PARETO output files), please pass a list of the custom tab names to get_data(). The following tabs are not standard PARETO inputs for the selected model type: ['Units', 'ProductionTanks', 'FNA', 'DesalinationTechnologies']
  [df_sets, df_parameters] = get_data(fpath, model_type="critical_mineral")


Along with network data the integrated model inputs a dictionary with the name of the site at which the desalination unit needs to be built and the corresponding number of evaporator stages that need to be built at that site. For example if at site "R01_IN" we want to build an MEE-MVR unit consisting of a single evaporator stage, the dictionary will be: {"R01":1}

For more information on MEE-MVR desalination please refer to the [mee_mvr_jupyter_notebook](../desalination_jupyter_notebooks/mee_mvr_jupyter_notebook.ipynb). 

In [3]:
desalination_dict = {"R01_IN": 1}

Now we will build the integrated model using the network data and the desalination dictionary

In [4]:
m = integrated_model_build(network_data, treatment_dict={"R01_IN": 1})

#### Initializing network ####
#### Initializing desalination unit ####
#### Build Complete #####


# Solving the integrated model
Now we import IPOPT - a non-linear programming solver from the `SolverFactory` available with Pyomo to solve the integrated model.

In [5]:
ipopt = pyo.SolverFactory("ipopt")
ipopt.options["mu_init"] = 1e-6
ipopt.options["bound_push"] = 1e-6
ipopt.solve(m, tee=True)

Ipopt 3.13.2: mu_init=1e-06
bound_push=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
 

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 7609, 'Number of variables': 7804, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 44.72001767158508}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

# Results
Now that the model is solved, we can obtain the results of our optimization problem. In this case we show how to obtain the various optimal network costs and optimal desalination CAPEX and OPEX

In [6]:
print("=================================================================")
print("#########            Costing Variables (USD)        #############")
print("=================================================================")
print("Piping cost")
print(pyo.value(m.m_network.arc_cost))
print("=================================================================")
print("Disposal cost")
print(pyo.value(m.m_network.disp_cost))
print("=================================================================")
print("Fresh water cost")
print(pyo.value(m.m_network.fresh_cost))
print("=================================================================")
print("Storage Cost")
print(pyo.value(m.m_network.stor_cost))
print("=================================================================")
print("Storage reward")
print(pyo.value(m.m_network.stor_rev))
print("=================================================================")
print("Treatment reward")
print(pyo.value(m.m_network.treat_rev))
print("=================================================================")
print("Treatment cost - OPEX")
print(
    1000
    * pyo.value(
        sum(
            sum(m.OPEX[s, t] / 365 * m.m_network.p_dt for t in m.m_network.s_T)
            for s in m.m_network.desalination_nodes
        )
    )
)
print("Treatment cost - CAPEX")
print(
    1000
    * pyo.value(
        sum(m.CAPEX[s] for s in m.m_network.desalination_nodes)
        / 365
        * m.m_network.p_dt
        * len(m.m_network.s_T)
    )
)
print("=================================================================")

#########            Costing Variables (USD)        #############
Piping cost
22308.28480466049
Disposal cost
1399182.1417958247
Fresh water cost
5402141.439497791
Storage Cost
1688572.3736907807
Storage reward
1671686.6499534836
Treatment reward
1155433.0840946478
Treatment cost - OPEX
473502.4210581018
Treatment cost - CAPEX
271348.4464275066


# Obtaining optimal MEE-MVR unit design
If the user wants to obtain the corresponding optimal desalination design, it can be done by using `backcalculate_mvr_design`

In [7]:
from pareto.models_extra.integrate_desal.integrated_models.backcalculate_mvr_design import (
    back_calculate_design,
)

In [8]:
back_calculate_design(m, treatment_dict=desalination_dict)

Desalination location - R01_IN
########################################################
Evaporator areas:
372.00000407598884
########################################################
Preheater area: 34.94438374200504
########################################################
Compressor capacity: 933.8398044634791
